# Toll Locations
This notebook creates the toll locations CSV based on [RD VIAL](https://rdvial.gob.do/peajes/) toll location data, scraped from the Leaflet map configuration.

In [1]:
import requests
from bs4 import BeautifulSoup as B
import json
import pandas as pd
import os

html = requests.get("https://rdvial.gob.do/peajes/")
cursor = B(html.text, "html.parser")
findings = cursor.find_all("div", id="wpmapblock_153ead85")  # Leaflet map DIV

# Mapping from RDVial toll names to MOPC toll names
title_map = {
    "Peaje 6 de Noviembre": "6 de Noviembre",
    "Estación de peaje Circunvalación Juan Bosch 1": "Tramo 1",
    "Estación de peaje Circunvalación Juan Bosch 2": "Tramo 2",
    "Estación de Peaje Circunvalación Santiago": "Santiago",
    "Peaje Coral 1": "Coral 1",
    "Peaje Coral 2": "Coral 2",
    "Peaje Duarte": "Duarte",
    "Peaje La Romana": "Romana",
    "Peaje Las Américas": "Americas",
    "Peaje Sánchez": "Sanchez",
    # "Peaje Azua": "Azua",
    "Peaje Marbella": "Marbella",
    "Peaje Naranjal": "Naranjal",
    "Peaje Guaraguao": "Guaraguao",
    "Peaje El Catey": "Catey"
}

Given the multiples properties needed to configure the markers in the Leaflet map, the only relevant information for toll visualization is the latitude and longitude properties; the title property is what we will use to join this table to the main toll DataFrame.

The last object added manually is toll data from Tramo 2B, which was not available in RD Vial, this was taken from [Peajes RD](https://peajesrd.com/peaje-circunvalacion-juan-bosch-tramo-2b/)

In [ ]:
settings_str = findings[0].get("data-settings")
settings_dict = json.loads(settings_str)
settings_dict = [{"lat": x["lat"], "lng": x["lng"], "title": title_map[x['title']]}
                 for x in settings_dict["map_marker"] if x['title'] in title_map.keys()]
settings_dict.append({
    'lat': 18.57225770188021,
    'lng': -69.77991741619846,
    'title': 'Tramo 2B'
})

df = pd.DataFrame(settings_dict).astype({"lat": float, "lng": float, "title": 'string'})
print(df.dtypes)
print(df.shape)
df.head()

Finally, the table is ready to be exported.

In [44]:
if not os.path.exists("./data"):
    os.makedirs("./data")
df.to_csv("./data/locations.csv", index=False)